In [2]:
import re

In [3]:
sc.master

'local[*]'

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
sc = spark.sparkContext

path = "/CA2/ProjectTweets.csv"

df = spark.read.csv(path)

In [43]:
df.tail(5)

[Row(_c0='1599995', _c1='2193601966', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='AmandaMarie1028', _c5='Just woke up. Having no school is the best feeling ever '),
 Row(_c0='1599996', _c1='2193601969', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='TheWDBoards', _c5='TheWDB.com - Very cool to hear old Walt interviews!  â\x99« http://blip.fm/~8bmta'),
 Row(_c0='1599997', _c1='2193601991', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='bpbabe', _c5='Are you ready for your MoJo Makeover? Ask me for details '),
 Row(_c0='1599998', _c1='2193602064', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='tinydiamondz', _c5='Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '),
 Row(_c0='1599999', _c1='2193602129', _c2='Tue Jun 16 08:40:50 PDT 2009', _c3='NO_QUERY', _c4='RyanTrevMorris', _c5='happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H ')]

In [44]:
df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [45]:
df = df.withColumnRenamed(df.columns[0],"row").withColumnRenamed(df.columns[1],"id").withColumnRenamed(df.columns[2],"date").withColumnRenamed(df.columns[3],"query").withColumnRenamed(df.columns[4],"handle").withColumnRenamed(df.columns[5],"tweet")

In [46]:
df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|row|        id|                date|   query|         handle|               tweet|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [47]:
from pyspark.sql.functions import col, regexp_replace, trim

In [50]:
# functions to apply to tweets

def rm_user(tweet):
    username = r'@\w+'
    tweet = regexp_replace(tweet,username,"")
    return tweet

def rm_http(tweet):
    hyperlink = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9A-fA-F][0-9a-fA-F]))+'
    tweet = regexp_replace(tweet, hyperlink,"")
    return tweet

def rm_spec_chars(tweet):
    pattern = r'[^a-zA-Z0-9\s]'
    tweet = regexp_replace(tweet, pattern,"")
    return tweet

def rm_whitespace(tweet):
    tweet = trim(regexp_replace(tweet, "\\s+", " "))
    return tweet

In [51]:
df = df.withColumn("tweet", rm_user(col("tweet")))

In [52]:
df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|row|        id|                date|   query|         handle|               tweet|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_| http://twitpic.c...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus| I dived many tim...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli| no, it's not beh...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf| not the whole crew |
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ| hey  long time n...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood| nope they did

In [53]:
df = df.withColumn("tweet", rm_http(col("tweet")))

In [54]:
df = df.withColumn("tweet", rm_spec_chars(col("tweet")))

In [55]:
df = df.withColumn("tweet", rm_whitespace(col("tweet")))

In [56]:
df.show()

+---+----------+--------------------+--------+---------------+--------------------+
|row|        id|                date|   query|         handle|               tweet|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|Awww thats a bumm...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|I dived many time...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|no its not behavi...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|  not the whole crew|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|          Need a hug|
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|hey long time no ...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|nope they didn

In [57]:
output_path = "/tweet_df.csv"

In [58]:
df.write.csv(output_path, header = True, mode = 'overwrite')